In [34]:
# initial setup

import pandas as pd
stock_closes = pd.read_csv('../data/raw/stock_data.csv')
stock_closes.set_index('Date', inplace=True)
days_12m = 252 # Approx 252 trading days in a year
days_3m = 63  # Approx 63 trading days in 3 months
days_1m = 21  # Approx 21 trading days in a month

In [76]:
# signal function to compute momentum scores based on z-scores

# 1. calculate z scores timeline for each stock (computes all for backtesting)
returns_12m = stock_closes.pct_change(periods=days_12m) # Pt - P(t-252) / P(t-252)
mean_12m = returns_12m.mean(axis=1) # across rows (all tickers at a single date)
std_12m = returns_12m.std(axis=1)
# print(mean_12m['2016-01-04'], std_12m) # ignore 2015 (all NaN)
z_scores_12m = (returns_12m.sub(mean_12m, axis=0)).div(std_12m, axis=0)
returns_3m = stock_closes.pct_change(periods=days_3m)
# ...
returns_1m = stock_closes.pct_change(periods=days_1m)
# ...

# 2. or if want latest scores only

'''
latest_returns_12m = returns_12m.iloc[-1] # last row
latest_mean_12m = latest_returns_12m.mean()
latest_std_12m = latest_returns_12m.std()
latest_z_scores_12m = (latest_returns_12m - latest_mean_12m) / latest_std_12m
'''

# returns = {returns_12m: stock_closes.pct_change(periods=days_12m), returns_3m: stock_closes.pct_change(periods=days_3m), returns_1m: stock_closes.pct_change(periods=days_1m)}
returns = [stock_closes.pct_change(periods=days_12m), stock_closes.pct_change(periods=days_3m), stock_closes.pct_change(periods=days_1m)]

dates = (returns[0].index[days_12m::days_1m]) # rebalancing dates (every month after first 12 months)

day = '2024-12-24' # example date
latest_z_scores = [] # list to hold latest z scores for each ticker, for each period (index 0 is 12m, 1 is 3m, 2 is 1m)

for period in returns:
    latest_returns = period.loc[day] # to get specific date's returns
    # latest_returns = returns.iloc[-1] # last row = that day's returns
    latest_mean = latest_returns.mean() # mean of last row
    latest_std = latest_returns.std() # std of last row
    latest_z_scores.append((latest_returns - latest_mean) / latest_std)
momentum_scores = (0.5 * latest_z_scores[0]) + (0.3 * latest_z_scores[1]) + (0.2 * latest_z_scores[2])
ranked_stocks = momentum_scores.sort_values(ascending=False)[:10] # top 10 momentum stocks to keep
top_tickers = ranked_stocks.index.tolist()
print(type(top_tickers))
for i in top_tickers:
    print(i)

<class 'list'>
NVDA
GOOGL
AMZN
META
GS
AAPL
JPM
DIS
BAC
BA
